In [1]:
from datasets import load_from_disk
#ds = load_dataset('klue', 'mrc')
ds = load_from_disk('mrc')

/root/miniconda3/envs/hf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained('llama-3.2-1b')
model = AutoModelForCausalLM.from_pretrained('llama-3.2-1b')

In [8]:
def preprocess_function(examples):
    inputs = [q + " " + c for q, c in zip(examples['question'], examples['context'])]
    model_inputs = tokenizer(
        inputs, 
        max_length=131072, 
        truncation=True,
        #padding='longest',
        return_attention_mask=True,
        return_token_type_ids=False
        )
    
    return model_inputs

tokenized_dataset = ds.map(
    preprocess_function, 
    batched=True,
    #remove_columns=ds.column_names,
    num_proc=4)


Map (num_proc=4): 100%|██████████| 5841/5841 [00:03<00:00, 1609.34 examples/s]


In [ ]:
tokenized_dataset['train'][0]

In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


In [6]:
model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable()

from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=500,
    evaluation_strategy='steps',
    eval_steps=500,
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=2,
    dataloader_num_workers=4,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)


/root/miniconda3/envs/hf/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/root/miniconda3/envs/hf/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [7]:
trainer.train()

ValueError: expected sequence of length 607 at dim 1 (got 925)